In [87]:
import numpy as np
import tensorflow as tf

In [88]:
#                  N  V C*T 
x = np.random.rand(2, 4, 5)
p = np.random.rand(5, 1)
A = tf.ones((2, 1, 1, 1))*tf.Variable(np.random.rand(3, 4, 4), dtype=tf.float32)

In [89]:
A

<tf.Tensor: id=5681, shape=(2, 3, 4, 4), dtype=float32, numpy=
array([[[[0.31466368, 0.14480618, 0.30655217, 0.9609877 ],
         [0.8802996 , 0.5074826 , 0.98072815, 0.71749896],
         [0.6729204 , 0.15679   , 0.77928454, 0.8573988 ],
         [0.8328191 , 0.22712839, 0.9309881 , 0.21164884]],

        [[0.03836715, 0.41943595, 0.4771193 , 0.99462974],
         [0.25554243, 0.18573959, 0.5424085 , 0.15370098],
         [0.5264157 , 0.9583813 , 0.1138834 , 0.8161457 ],
         [0.194951  , 0.8009568 , 0.05754864, 0.8958153 ]],

        [[0.4034327 , 0.8244697 , 0.12411056, 0.2622539 ],
         [0.7077735 , 0.22267327, 0.7393886 , 0.4303404 ],
         [0.53576523, 0.394839  , 0.83175516, 0.53424203],
         [0.32423282, 0.89419055, 0.62233686, 0.23237012]]],


       [[[0.31466368, 0.14480618, 0.30655217, 0.9609877 ],
         [0.8802996 , 0.5074826 , 0.98072815, 0.71749896],
         [0.6729204 , 0.15679   , 0.77928454, 0.8573988 ],
         [0.8328191 , 0.22712839, 0.9309881 

In [90]:
y = tf.matmul(x, tf.math.l2_normalize(p))
indices = tf.argsort(y, axis=-2, direction='DESCENDING')
indices = indices[:, :3]
y_hat = tf.math.sigmoid(tf.gather_nd(y, indices, batch_dims=1))
x_l = tf.gather_nd(x, indices, batch_dims=1)
x_l = tf.multiply(x_l, y_hat)

In [91]:
A = tf.transpose(A, perm=[0, 2, 3, 1])

In [94]:
A_l = tf.gather(tf.gather(A, tf.squeeze(indices, axis=-1), batch_dims=1),  tf.squeeze(indices, axis=-1), axis=-2, batch_dims=1)
A_l = tf.transpose(A_l, perm=[0, 3, 1, 2])
A_l

<tf.Tensor: id=6205, shape=(2, 3, 3, 3), dtype=float32, numpy=
array([[[[0.31466368, 0.30655217, 0.14480618],
         [0.6729204 , 0.77928454, 0.15679   ],
         [0.8802996 , 0.98072815, 0.5074826 ]],

        [[0.03836715, 0.4771193 , 0.41943595],
         [0.5264157 , 0.1138834 , 0.9583813 ],
         [0.25554243, 0.5424085 , 0.18573959]],

        [[0.4034327 , 0.12411056, 0.8244697 ],
         [0.53576523, 0.83175516, 0.394839  ],
         [0.7077735 , 0.7393886 , 0.22267327]]],


       [[[0.31466368, 0.30655217, 0.9609877 ],
         [0.6729204 , 0.77928454, 0.8573988 ],
         [0.8328191 , 0.9309881 , 0.21164884]],

        [[0.03836715, 0.4771193 , 0.99462974],
         [0.5264157 , 0.1138834 , 0.8161457 ],
         [0.194951  , 0.05754864, 0.8958153 ]],

        [[0.4034327 , 0.12411056, 0.2622539 ],
         [0.53576523, 0.83175516, 0.53424203],
         [0.32423282, 0.62233686, 0.23237012]]]], dtype=float32)>

In [ ]:
y = tf.matmul(x, tf.math.l2_normalize(p))
indices = tf.argsort(y, axis=-2, direction='DESCENDING')
indices = indices[:, :, :2]
y_hat = tf.math.sigmoid(tf.gather_nd(y, indices, batch_dims=2))
x_l = tf.gather_nd(x, indices, batch_dims=2)
x_l = tf.multiply(x_l, y_hat)
A_l = tf.gather(tf.gather(A, tf.squeeze(indices, axis=-1), batch_dims=1),  tf.squeeze(indices, axis=-1), axis=-1, batch_dims=2)

In [ ]:
A_l

In [ ]:
class GPool(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    # input_shape: N  T  V  C
    def build(self, input_shape, k):
        self.p = self.add_weight(name="projection_vector",
                                 shape=[input_shape[-1], 1],
                                 trainable=True)
        self.k = k

    def call(self, x, A):
        # project onto p
        y = tf.matmul(x, tf.math.l2_normalize(self.p))

        #rank the projections
        indices = tf.argsort(y, axis=-2, direction='DESCENDING')
        indices = indices[:, :, :tf.cast(self.k*tf.shape(A)[-1], tf.uint8)]

        # get y at indices and apply sigmoid
        y_hat = tf.math.sigmoid(tf.gather_nd(y, indices, batch_dims=2))

        # get x at indices and apply y_hat
        x = tf.gather_nd(x, indices, batch_dims=2)
        x = tf.multiply(x, y_hat)

        # get indices rows and cols
        indices = tf.squeeze(indices, axis=-1)
        A = tf.gather(A, indices)
        A = tf.gather(A, indices, axis=-1, batch_dims=2)

        return x, A